In [1]:
import json
import logging
import string
import re
from collections import defaultdict, OrderedDict

import scrapy
from scrapy.crawler import CrawlerProcess
from tqdm import tqdm

OUTPUT_FILE = 'mayo-diseases_full.json'
diseases = []


class MayoClinicSpider(scrapy.Spider):
    """
    This spider extracts the information of every disease published at
    https://www.mayoclinic.org/diseases-conditions/index
    Each disease, cause and risk factor is uniquely identified by an id the first time is encountered. For example,
    after crawling this page:
    https://www.mayoclinic.org/diseases-conditions/abdominal-aortic-aneurysm/symptoms-causes/syc-20350688
    The disease 'Abdominal aortic aneurysm' is given the id '1'. This diseases is then linked with the causes
    'Tobacco use' (id '1'), 'atherosclerosis' (id '2'), 'High blood pressure' (id '3'), etc. The same is done for the
    risk factors.
    In this way, if after crawling another disease, the cause 'Heredity' is encountered again, this new disease will be
    linked with the cause id '1'.
    In the end, a JSON Array is generated. Every element of this array have the following format::
        {
        "disease_id":1,
        "disease_name": "Abdominal aortic aneurysm",
        "causes": [ { "cause_id": 1, "cause_name": "Tobacco use" }, { "cause_id":2, "cause_name": "Atherosclerosis" } ],
        "risk_factors": [ { "risk_id":1, "risk_name":"Age"}, { "risk_id": 2, "risk_name": "Tobacco use" } ]
        }
    """
    name = 'mayo_clinic_spider'
    
    start_urls = []
    for s in string.ascii_uppercase :
        start_urls.append('https://www.mayoclinic.org/diseases-conditions/index?letter='+str(s))

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        logging.getLogger('scrapy').setLevel(logging.WARNING)
        self.diseases = []
        self.diseases_id = defaultdict(lambda: len(self.diseases_id))
        self.causes_id = defaultdict(lambda: len(self.causes_id))
        self.risks_id = defaultdict(lambda: len(self.risks_id))

    def parse(self, response):
        """Find every link on the index page that links to a 'disease' page"""
        links = response.xpath('//li/a[re:test(@href, "/diseases-conditions/.*/symptoms-causes")]/@href').extract()
        with tqdm(total=len(links)) as pbar:
            for link in links:
                disease_name = re.match('/diseases-conditions/(.*)/symptoms-causes', link).group(1)
                pbar.set_description("Processing {}".format(disease_name))
                pbar.update(1)
                disease_page = response.urljoin(link)
                yield scrapy.Request(disease_page, callback=self.parse_disease)

    def parse_disease(self, response):
        """Extract the information from the 'disease' page"""
        disease_name = response.xpath('//h1/a/text()').extract()[0]
        causes_names = response.xpath('//h2[text()="Causes"]/following-sibling::ul[1]/li/strong/text()').extract()
        risks_names = response.xpath('//h2[text()="Risk factors"]/following-sibling::ul[1]/li/strong/text()').extract()
        diseases.append(
            OrderedDict([
                ('disease_id', self.diseases_id[disease_name]),
                ('disease_name', disease_name),
                ('causes', [
                    OrderedDict([
                        ('cause_id', self.causes_id[cause]),
                        ('cause_name', cause)
                    ])
                    for cause in map(lambda cause: cause[:-1], causes_names)
                ]),
                ('risk_factors', [
                    OrderedDict([
                        ('risk_id', self.risks_id[risk]),
                        ('risk_name', risk)
                    ])
                    for risk in map(lambda risk: risk[:-1], risks_names)
                ])
            ])
        )


process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(MayoClinicSpider)
process.start()

with open(OUTPUT_FILE, 'wt') as out:
    json.dump(diseases, out, indent=3)

2021-06-01 20:51:03 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2021-06-01 20:51:03 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.2.0, Python 3.7.7 (default, May  6 2020, 11:45:54) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 20.0.1 (OpenSSL 1.1.1k  25 Mar 2021), cryptography 3.4.4, Platform Windows-10-10.0.19041-SP0
2021-06-01 20:51:03 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-06-01 20:51:04 [scrapy.crawler] INFO: Overridden settings:
{'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2021-06-01 20:51:04 [scrapy.extensions.telnet] INFO: Telnet Password: 16aadb2e1fa8837c
2021-06-01 20:51:05 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
Processing h-pylori:   0%|          | 0/150 [00:00<?, ?it/s]

  0%|         

Processing viral-gastroenteritis:  22%|██▏       | 13/58 [00:01<00:03, 13.78it/s]

Processing viral-gastroenteritis:  24%|██▍       | 14/58 [00:01<00:03, 13.38it/s]


Processing cervical-cancer:   5%|▍         | 12/259 [00:01<00:17, 13.81it/s] 


Processing chronic-lymphocytic-leukemia:   5%|▌         | 13/259 [00:01<00:17, 13.81it/s]


Processing chronic-lymphocytic-leukemia:   5%|▌         | 14/259 [00:01<00:18, 13.44it/s]



Processing acl-injury:   5%|▌         | 12/221 [00:00<00:14, 14.02it/s]        



Processing acne:   6%|▌         | 13/221 [00:00<00:14, 14.02it/s]      



Processing acne:   6%|▋         | 14/221 [00:00<00:15, 13.53it/s]




Processing gastroparesis:  11%|█         | 12/111 [00:00<00:07, 14.07it/s]      




Processing delayed-sleep-phase:  12%|█▏        | 13/111 [00:00<00:06, 14.07it/s]




Processing delayed-sleep-phase:  13%|█▎        | 14/111 [00:00<00:07, 13.50it/s]





Processing febrile-seizure:  15%|█▌        | 12/78 [00:00<00:04, 14.35it/s]





Pro

Processing dislocation:  40%|████      | 4/10 [00:00<00:00,  7.54it/s]

















  0%|          | 0/42 [00:00<?, ?it/s]

















Processing vaginal-agenesis:   0%|          | 0/42 [00:00<?, ?it/s]















Processing childhood-obesity:   2%|▏         | 1/46 [00:00<00:13,  3.28it/s]















Processing childhood-obesity:   4%|▍         | 2/46 [00:00<00:06,  6.50it/s]















Processing obsessive-compulsive-disorder:   4%|▍         | 2/46 [00:00<00:06,  6.50it/s]





Processing cold-sore:  26%|██▌       | 20/78 [00:01<00:05,  9.67it/s]





Processing valley-fever:  27%|██▋       | 21/78 [00:01<00:05,  9.67it/s]





Processing valley-fever:  28%|██▊       | 22/78 [00:01<00:06,  8.32it/s]
















Processing popliteal-artery-entrapment:   0%|          | 1/219 [00:00<01:05,  3.33it/s]
















Processing popliteal-artery-entrapment:   1%|          | 2/219 [00:00<00:32,  6.61it/s]
















Processing pagets-disease-of-bone:   1%|     

Processing ice-cream-headaches:  18%|█▊        | 27/150 [00:02<00:19,  6.42it/s]           

Processing rubella:  45%|████▍     | 26/58 [00:02<00:04,  6.64it/s]         

Processing rubella:  47%|████▋     | 27/58 [00:02<00:04,  6.42it/s]

Processing gestational-diabetes:  47%|████▋     | 27/58 [00:02<00:04,  6.42it/s]


Processing leukemia:  10%|█         | 26/259 [00:02<00:34,  6.66it/s]     


Processing leukemia:  10%|█         | 27/259 [00:02<00:36,  6.40it/s]


Processing mouth-cancer:  10%|█         | 27/259 [00:02<00:36,  6.40it/s]



Processing acute-liver-failure:  11%|█▏        | 25/221 [00:02<00:35,  5.55it/s]



Processing acute-liver-failure:  12%|█▏        | 26/221 [00:02<00:34,  5.66it/s]



Processing acute-lymphocytic-leukemia:  12%|█▏        | 26/221 [00:02<00:34,  5.66it/s]




Processing depression:  23%|██▎       | 26/111 [00:02<00:12,  6.70it/s]                              




Processing depression:  24%|██▍       | 27/111 [00:02<00:13,  6.40it/s]















Processing pulmonary-fibrosis:  40%|███▉      | 31/78 [00:03<00:07,  6.04it/s]       
















Processing bells-palsy:   5%|▌         | 11/219 [00:02<00:36,  5.75it/s]   
















Processing bells-palsy:   5%|▌         | 12/219 [00:02<00:34,  6.01it/s]
















Processing pancreatitis:   5%|▌         | 12/219 [00:02<00:34,  6.01it/s]









Processing anal-itching:  21%|██        | 15/71 [00:02<00:09,  5.97it/s]









Processing anal-itching:  23%|██▎       | 16/71 [00:02<00:09,  6.09it/s]









Processing rectal-prolapse:  23%|██▎       | 16/71 [00:02<00:09,  6.09it/s]













Processing pseudotumor-cerebri:  16%|█▋        | 12/73 [00:02<00:11,  5.50it/s]     













Processing pseudotumor-cerebri:  18%|█▊        | 13/73 [00:02<00:10,  5.76it/s]













Processing illness-anxiety-disorder:  18%|█▊        | 13/73 [00:02<00:10,  5.76it/s]










Processing left-ventricular-hypertrophy:  16%|█▌        | 12/77 [00:02<00:10,  6.01it/s]








...

Processing chronic-myelogenous-leukemia:  51%|█████     | 132/259 [00:13<00:05, 23.34it/s]



Processing sprained-ankle:  62%|██████▏   | 136/221 [00:13<00:03, 25.31it/s]



Processing sprained-ankle:  62%|██████▏   | 137/221 [00:13<00:03, 26.85it/s]











Processing age-spots:  57%|█████▋    | 116/205 [00:11<00:03, 25.27it/s]        











Processing age-spots:  57%|█████▋    | 117/205 [00:11<00:03, 27.08it/s]











Processing rectal-ulcer:  57%|█████▋    | 117/205 [00:11<00:03, 27.08it/s]
















Processing polycystic-kidney-disease:  53%|█████▎    | 116/219 [00:11<00:03, 28.09it/s]
















Processing polycystic-kidney-disease:  53%|█████▎    | 117/219 [00:11<00:03, 29.12it/s]
















Processing hypoparathyroidism:  95%|█████████▌| 143/150 [00:13<00:00, 25.46it/s]


Processing copd:  51%|█████▏    | 133/259 [00:13<00:05, 23.34it/s]                        


Processing chronic-pelvic-pain:  52%|█████▏    | 134/259 [00:13<00:05, 23.34it/s]



Proc

Processing sexually-transmitted-diseases-stds:  70%|███████   | 144/205 [00:11<00:01, 31.43it/s]
















Processing dresslers-syndrome:  64%|██████▍   | 141/219 [00:12<00:02, 29.94it/s]    
















Processing postpartum-depression:  65%|██████▍   | 142/219 [00:12<00:02, 29.94it/s]
















Processing sheehans-syndrome:  65%|██████▌   | 143/219 [00:12<00:02, 29.94it/s]    


Processing common-cold:  61%|██████    | 158/259 [00:14<00:03, 26.30it/s]   


Processing common-cold:  61%|██████▏   | 159/259 [00:14<00:02, 34.01it/s]


Processing colic:  61%|██████▏   | 159/259 [00:14<00:02, 34.01it/s]      



Processing canker-sore:  73%|███████▎  | 161/221 [00:14<00:02, 25.75it/s]                



Processing canker-sore:  73%|███████▎  | 162/221 [00:14<00:01, 31.15it/s]











Processing progressive-supranuclear-palsy:  70%|███████   | 144/205 [00:12<00:01, 31.43it/s]    











Processing pcos:  71%|███████   | 145/205 [00:12<00:01, 31.43it/s]                

Processing arteriovenous-malformation:  79%|███████▊  | 174/221 [00:14<00:01, 32.66it/s]











Processing scorpion-stings:  76%|███████▌  | 156/205 [00:12<00:01, 33.02it/s] 











Processing stomach-cancer:  77%|███████▋  | 157/205 [00:12<00:01, 33.02it/s] 











Processing viral-gastroenteritis:  77%|███████▋  | 158/205 [00:12<00:01, 33.02it/s]
















Processing preeclampsia:  71%|███████   | 156/219 [00:12<00:01, 32.70it/s]                  
















Processing preeclampsia:  72%|███████▏  | 157/219 [00:12<00:01, 32.35it/s]


Processing common-variable-immunodeficiency:  67%|██████▋   | 174/259 [00:14<00:02, 37.97it/s]


Processing common-variable-immunodeficiency:  68%|██████▊   | 175/259 [00:14<00:02, 37.97it/s]


Processing common-warts:  68%|██████▊   | 176/259 [00:14<00:02, 37.97it/s]                    



Processing giant-cell-arteritis:  79%|███████▊  | 174/221 [00:14<00:01, 32.66it/s]      



Processing takayasus-arteritis:  79%|███████▉  | 17

Processing primary-progressive-aphasia:  83%|████████▎ | 182/219 [00:13<00:00, 37.05it/s]


Processing contact-dermatitis:  78%|███████▊  | 202/259 [00:15<00:01, 40.06it/s]      


Processing convergence-insufficiency:  78%|███████▊  | 203/259 [00:15<00:01, 40.06it/s]


Processing thalassemia:  79%|███████▉  | 204/259 [00:15<00:01, 40.06it/s]              



Processing athletes-foot:  91%|█████████▏| 202/221 [00:15<00:00, 38.00it/s]                   



Processing atopic-dermatitis-eczema:  92%|█████████▏| 203/221 [00:15<00:00, 38.00it/s]



Processing atrial-fibrillation:  92%|█████████▏| 204/221 [00:15<00:00, 38.00it/s]     











Processing progressive-supranuclear-palsy:  91%|█████████ | 186/205 [00:13<00:00, 36.93it/s]











Processing supraventricular-tachycardia:  91%|█████████ | 187/205 [00:13<00:00, 36.93it/s]  











Processing supraventricular-tachycardia:  92%|█████████▏| 188/205 [00:13<00:00, 39.59it/s]
















Processing primary-sclerosing-cholang

Processing cyclothymia:  94%|█████████▍| 243/259 [00:16<00:00, 50.79it/s]


Processing bartholin-cyst:  94%|█████████▍| 244/259 [00:16<00:00, 50.79it/s]


Processing ganglion-cyst:  95%|█████████▍| 245/259 [00:16<00:00, 50.79it/s] 


Processing kidney-cysts:  95%|█████████▍| 246/259 [00:16<00:00, 50.79it/s] 


Processing ovarian-cysts:  95%|█████████▌| 247/259 [00:16<00:00, 50.79it/s]


Processing pancreatic-cysts:  96%|█████████▌| 248/259 [00:16<00:00, 50.79it/s]


Processing pancreatic-cysts:  96%|█████████▌| 249/259 [00:16<00:00, 74.03it/s]


Processing pilonidal-cyst:  96%|█████████▌| 249/259 [00:16<00:00, 74.03it/s]  


Processing spermatocele:  97%|█████████▋| 250/259 [00:16<00:00, 74.03it/s]  


Processing cystic-fibrosis:  97%|█████████▋| 251/259 [00:16<00:00, 74.03it/s]


Processing cystitis:  97%|█████████▋| 252/259 [00:16<00:00, 74.03it/s]       


Processing interstitial-cystitis:  98%|█████████▊| 253/259 [00:16<00:00, 74.03it/s]


Processing cystocele:  98%|█████████▊| 254

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements'


ModuleNotFoundError: No module named 'neo4j.v1'